# Example of Data Processing with Python
## Download
In Python there is no direct API to the call report data, so we implemented a semi-automatic web crawler. For this
purpose, we used the functionalities of the Selenium library which simulates the use of a browser.To achieve this, a
driver must be installed in advance which provides an interface to the desired browser.
Depending on the browser you want to use later, you choose the corresponding a driver. A list of drivers can be found
on the homepage of the library. Beforehand, it is important to inspect the HTML structure of the form to define how the 
form should be handled 


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import numpy as np
import os
import time
import glob
import zipfile


# Helper function to keep the browser open until the file has been downloaded
def download_wait(directory, timeout, nfiles=None):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds


path = os.path.dirname(__file__)
data_path = os.path.join(path, "..", "data")
download_path = os.path.join(path, "..", "..", "Downloads")
url = "https://cdr.ffiec.gov/public/PWS/DownloadBulkData.aspx"
years = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", 
         "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", 
         "2017", "2018", "2019", "2020"]

for year in years:

    periods = ["03/31/" + year, "06/30/" + year, "09/30/" + year, "12/31/" + year]

    # Initialize the browser and call the url
    driver = webdriver.Chrome()
    driver.get(url)

    try:
        element = WebDriverWait(driver, 200).until(EC.presence_of_element_located((By.TAG_NAME, "form")))

        selectProduct = Select(driver.find_element_by_id("ListBox1"))
        selectProduct.select_by_value("ReportingSeriesSinglePeriod")

        for period in periods:
            selectYear = Select(driver.find_element_by_id("DatesDropDownList"))
            selectYear.select_by_visible_text(period)

            driver.find_element_by_id("Download_0").click()

        driver.implicitly_wait(download_wait(download_path, 40))
    finally:
        driver.quit()

    # Get latest latest filename with path
    list_of_files = glob.glob(os.path.join(download_path, "*"))

    # Convert list to np array
    list_of_files = np.array(list_of_files)

    # Get latest latest filename with path
    list_of_files = sorted(glob.glob(os.path.join(download_path, "*.zip")))

    list_of_files = np.array(list_of_files)

    for file_path in list_of_files[0:5]:
        # Extract the filename from path
        file = os.path.basename(os.path.normpath(file_path))

        # Create destination path to move the file
        data_move_path = os.path.join(data_path, file)

        # Move the file from downloads directory to destination
        os.replace(file_path, data_move_path)

        # Unzip downloaded file
        z = zipfile.ZipFile(data_move_path)
        z.extractall(os.path.join(data_move_path, "..", "Call_Report_" + year))
        z.close()

        # Delete zipped file
        os.remove(data_move_path)



## Process
test

In [ ]:
import pandas as pd
import os
from datetime import datetime

path = os.path.dirname(__file__)

years = ["2015", "2016", "2017", "2018", "2019", "2020"]
quarters = ["0331", "0630", "0930", "1231"]
concat_column = ["IDRSSD"]
por_columns = ["FDIC Certificate Number", "OTS Docket Number", "Primary ABA Routing Number",
               "Financial Institution Name", "Financial Institution City", "Financial Institution State",
               "Financial Institution Filing Type"]
rc_columns = ["RCFD2170", "RCFD2948", "RCFDG105", "RCON2170", "RCON2948", "RCONG105"]
ri_columns = ["RIAD4340", "RIAD4107", "RIAD4073", "RIAD4079", "RIAD4093"]
rcr_columns = ["RCFA7204", "RCFA7205", "RCOA7204", "RCOA7205"]
new_columns = ["period", "year", "quarter"]

master_df = pd.DataFrame(columns=concat_column + por_columns + rc_columns + ri_columns + rcr_columns + new_columns)

for year in years:
    year_data_path = os.path.join(path, "..", "data", "Call_Report_" + year)
    print(year)

    for index, quarter in enumerate(quarters):

        if (year == "2020") and (quarter == "1231"):
            continue

        datetimeobject = datetime.strptime(quarter + year, '%m%d%Y')

        por_file_path = os.path.join(year_data_path, "FFIEC CDR Call Bulk POR " + quarter + year + ".txt")
        rc_file_path = os.path.join(year_data_path, "FFIEC CDR Call Schedule RC " + quarter + year + ".txt")
        ri_file_path = os.path.join(year_data_path, "FFIEC CDR Call Schedule RI " + quarter + year + ".txt")
        rcr_file_path = os.path.join(year_data_path, "FFIEC CDR Call Schedule RCRI " + quarter + year + ".txt")

        df_por = pd.read_csv(por_file_path, sep='\t', usecols=concat_column + por_columns, low_memory=False)
        df_rc = pd.read_csv(rc_file_path, sep='\t', low_memory=False, usecols=concat_column + rc_columns, skiprows=[1])
        df_ri = pd.read_csv(ri_file_path, sep='\t', low_memory=False, usecols=concat_column + ri_columns, skiprows=[1])
        df_rcr = pd.read_csv(rcr_file_path, sep='\t', low_memory=False, usecols=concat_column + rcr_columns, skiprows=[1])

        buffer_df = pd.merge(df_por, df_rc, how='inner', on='IDRSSD')
        buffer_df = pd.merge(buffer_df, df_ri, how='inner', on='IDRSSD')
        buffer_df = pd.merge(buffer_df, df_rcr, how='inner', on='IDRSSD')

        buffer_df[new_columns[0]] = datetimeobject.strftime('%Y-%m-%d')
        buffer_df[new_columns[1]] = year
        buffer_df[new_columns[2]] = index+1

        master_df = pd.concat([master_df, buffer_df], ignore_index=True, sort=True)


master_df['RCFD_Return_on_Assets'] = master_df['RIAD4340'] / master_df['RCFD2170']
master_df['RCON_Return_on_Assets'] = master_df['RIAD4340'] / master_df['RCON2170']
master_df['RCFD_Equity_Ratio'] = master_df['RCFDG105'] / master_df['RCFD2170']
master_df['RCON_Equity_Ratio'] = master_df['RCONG105'] / master_df['RCON2170']

master_df['RCFD_Return_on_Assets'].fillna(master_df['RCON_Return_on_Assets'], inplace=True)
master_df['RCFD_Equity_Ratio'].fillna(master_df['RCON_Equity_Ratio'], inplace=True)
master_df['RCFD2170'].fillna(master_df['RCON2170'], inplace=True)
master_df['RCFD2948'].fillna(master_df['RCON2948'], inplace=True)
master_df['RCFDG105'].fillna(master_df['RCONG105'], inplace=True)
master_df['RCFA7204'].fillna(master_df['RCOA7204'], inplace=True)
master_df['RCFA7205'].fillna(master_df['RCOA7205'], inplace=True)


master_df.drop(['RCON_Return_on_Assets', 'RCON_Equity_Ratio', 'RCON2170', 'RCON2948', 'RCONG105', 'RCOA7204', 'RCOA7205'],
               axis=1, inplace=True)
master_df.rename(columns={'RCFD_Return_on_Assets': 'Return_on_Assets', 'RCFD_Equity_Ratio': 'Equity_Ratio',
                          'RCFD2170': 'Total_Assets', 'RCFD2948': 'Total_Liabilities', 'RCFDG105': 'Total_Equity_Capital',
                          'RCFA7204': 'Leverage_Ratio', 'RCFA7205': 'Total_Capital_Ratio', 'RIAD4340': 'Net_Income',
                          'RIAD4107': 'Total_Interest_Income', 'RIAD4073': 'Total_Interest_Expense',
                          'RIAD4079': 'Total_Noninterest_Income', 'RIAD4093': 'Total_Noninterest_Expense'},
                 inplace=True)


master_data_path = os.path.join(path, "..", "data", "master_cr_file_15-20.txt")
master_df.to_csv(master_data_path, sep='\t', index=False)

## Visualization

In [1]:

import os
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

path = os.path.dirname(__file__)
data_path = os.path.join(path, "..", "data", "master_cr_file_15-20.txt")
df = pd.read_csv(data_path, sep='\t', low_memory=False)

new_df = df.loc[(df['year'] == 2018) & (df['quarter'] == 3)]

bank_count_relative = new_df['Financial Institution State'].value_counts(normalize=True)
bank_count_absolute = new_df['Financial Institution State'].value_counts()

ax1 = sns.barplot(x=bank_count_absolute.index, y=bank_count_absolute.values)
ax1.set_title("Absolute Number of Banks by state for the 3rd quarter of 2018")
ax1.set_ylabel('Number of Banks')
ax1.set_xlabel('States')

ax2 = sns.barplot(x=bank_count_relative.index, y=bank_count_relative.values)
ax2.set_title("Relative Number of Banks by state for the 3rd quarter of 2018")
ax2.set_ylabel('Number of Banks')
ax2.set_xlabel('States')

NameError: name '__file__' is not defined